# Test RAY version of PDF2Parquet

In [1]:
## Setup env
## we are installing 1.0.0.a4

#!pip install data-prep-toolkit-transforms[ray,all]==1.0.0a4

In [2]:
## download files

INPUT_DIR = 'input2'

import shutil
shutil.os.makedirs( INPUT_DIR, exist_ok=True)

!wget 'https://arxiv.org/pdf/1706.03762' -O '{INPUT_DIR}/attention.pdf'
!wget 'https://arxiv.org/pdf/2405.04324'  -O '{INPUT_DIR}/granite.pdf'
!wget 'https://arxiv.org/pdf/2405.04324'  -O '{INPUT_DIR}/granite2.pdf'

--2025-01-19 22:20:32--  https://arxiv.org/pdf/1706.03762
Resolving arxiv.org (arxiv.org)... 151.101.131.42, 151.101.195.42, 151.101.3.42, ...
Connecting to arxiv.org (arxiv.org)|151.101.131.42|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2215244 (2.1M) [application/pdf]
Saving to: ‘input2/attention.pdf’

input2/attention.pd 100%[===================>]   2.11M  1.79MB/s    in 1.2s    

2025-01-19 22:20:34 (1.79 MB/s) - ‘input2/attention.pdf’ saved [2215244/2215244]

--2025-01-19 22:20:34--  https://arxiv.org/pdf/2405.04324
Resolving arxiv.org (arxiv.org)... 151.101.3.42, 151.101.195.42, 151.101.131.42, ...
Connecting to arxiv.org (arxiv.org)|151.101.3.42|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1273644 (1.2M) [application/pdf]
Saving to: ‘input2/granite.pdf’

input2/granite.pdf  100%[===================>]   1.21M  2.21MB/s    in 0.5s    

2025-01-19 22:20:35 (2.21 MB/s) - ‘input2/granite.pdf’ saved [1273644/1273644]

--2025-0

In [3]:
%%time 

from dpk_pdf2parquet.ray.transform import Pdf2Parquet
from data_processing.utils import GB
from dpk_pdf2parquet.transform import pdf2parquet_contents_types


result = Pdf2Parquet(input_folder= INPUT_DIR,
                    output_folder= "output_pdf2pq", 
                    data_files_to_use=['.pdf'],
                    pdf2parquet_contents_type=pdf2parquet_contents_types.MARKDOWN,
                    # pdf2parquet_contents_type=pdf2parquet_contents_types.JSON,
                    
                    ## runtime options
                    run_locally= True,
                    num_cpus=  0.5, 
                    memory= 2 * GB, 
                    runtime_num_workers = 2, ## Note: has to be one for this particular job, to prevent race condition when downloading models!
               ).transform()

if result == 0:
    print (f"✅ Completed successfully")
else:
    raise Exception (f"❌ Failed")

22:20:41 INFO - pdf2parquet parameters are : {'batch_size': -1, 'artifacts_path': None, 'contents_type': <pdf2parquet_contents_types.MARKDOWN: 'text/markdown'>, 'do_table_structure': True, 'do_ocr': True, 'ocr_engine': <pdf2parquet_ocr_engine.EASYOCR: 'easyocr'>, 'bitmap_area_threshold': 0.05, 'pdf_backend': <pdf2parquet_pdf_backend.DLPARSE_V2: 'dlparse_v2'>, 'double_precision': 8}
22:20:41 INFO - pipeline id pipeline_id
22:20:41 INFO - code location None
22:20:41 INFO - number of workers 2 worker options {'num_cpus': 0.5, 'memory': 2147483648, 'max_restarts': -1}
22:20:41 INFO - actor creation delay 0
22:20:41 INFO - job details {'job category': 'preprocessing', 'job name': 'pdf2parquet', 'job type': 'ray', 'job id': 'job_id'}
22:20:41 INFO - data factory data_ is using local data access: input_folder - input2 output_folder - output_pdf2pq
22:20:41 INFO - data factory data_ max_files -1, n_sample -1
22:20:41 INFO - data factory data_ Not using data sets, checkpointing False, max files

(RayTransformFileProcessor pid=1146872) ERR#: COULD NOT CONVERT TO RS THIS TABLE TO COMPUTE SPANS


(orchestrate pid=1145981) 22:21:45 INFO - Completed 1 files in 0.813 min
(orchestrate pid=1145981) 22:21:45 INFO - Completed 1 files (33.333%)  in 0.813 min. Waiting for completion
(RayTransformFileProcessor pid=1146872) 22:20:49 INFO - Initializing models
Fetching 9 files: 100%|██████████| 9/9 [00:00<00:00, 62914.56it/s]
(RayTransformFileProcessor pid=1146872) Neither CUDA nor MPS are available - defaulting to CPU. Note: This module is much faster with a GPU.
(orchestrate pid=1145981) 22:24:28 INFO - Completed processing 3 files in 3.53 min
(orchestrate pid=1145981) 22:24:28 INFO - done flushing in 0.001 sec
22:24:38 INFO - Completed execution in 3.957 min, execution result 0


✅ Completed successfully
CPU times: user 4.98 s, sys: 1.37 s, total: 6.35 s
Wall time: 4min 2s
